In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import os
# os.chdir("/content/drive/My Drive/Colab Notebooks/")

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1-movielens_sample数据集" data-toc-modified-id="1-movielens_sample数据集-1">1 movielens_sample数据集</a></span><ul class="toc-item"><li><span><a href="#1.1-数据载入" data-toc-modified-id="1.1-数据载入-1.1">1.1 数据载入</a></span></li><li><span><a href="#1.2-数据预处理" data-toc-modified-id="1.2-数据预处理-1.2">1.2 数据预处理</a></span><ul class="toc-item"><li><span><a href="#1.2.1-标签数值化" data-toc-modified-id="1.2.1-标签数值化-1.2.1">1.2.1 标签数值化</a></span></li><li><span><a href="#1.2.2-生成词向量" data-toc-modified-id="1.2.2-生成词向量-1.2.2">1.2.2 生成词向量</a></span></li></ul></li><li><span><a href="#1.3-拆分数据集" data-toc-modified-id="1.3-拆分数据集-1.3">1.3 拆分数据集</a></span></li><li><span><a href="#1.4-使用DeepFM训练" data-toc-modified-id="1.4-使用DeepFM训练-1.4">1.4 使用DeepFM训练</a></span><ul class="toc-item"><li><span><a href="#1.4.1-实例化模型" data-toc-modified-id="1.4.1-实例化模型-1.4.1">1.4.1 实例化模型</a></span></li><li><span><a href="#1.4.2-编译模型" data-toc-modified-id="1.4.2-编译模型-1.4.2">1.4.2 编译模型</a></span></li><li><span><a href="#1.4.3-训练模型" data-toc-modified-id="1.4.3-训练模型-1.4.3">1.4.3 训练模型</a></span></li></ul></li></ul></li><li><span><a href="#2-movielens数据集" data-toc-modified-id="2-movielens数据集-2">2 movielens数据集</a></span><ul class="toc-item"><li><span><a href="#2.1-读入数据" data-toc-modified-id="2.1-读入数据-2.1">2.1 读入数据</a></span></li><li><span><a href="#2.2-预处理" data-toc-modified-id="2.2-预处理-2.2">2.2 预处理</a></span><ul class="toc-item"><li><span><a href="#2.2.1-标签数值化" data-toc-modified-id="2.2.1-标签数值化-2.2.1">2.2.1 标签数值化</a></span></li><li><span><a href="#2.2.2生成词向量" data-toc-modified-id="2.2.2生成词向量-2.2.2">2.2.2生成词向量</a></span></li></ul></li><li><span><a href="#2.3-拆分数据集并为模型生成输入数据" data-toc-modified-id="2.3-拆分数据集并为模型生成输入数据-2.3">2.3 拆分数据集并为模型生成输入数据</a></span></li><li><span><a href="#2.4-用DeepFM建模" data-toc-modified-id="2.4-用DeepFM建模-2.4">2.4 用DeepFM建模</a></span></li><li><span><a href="#2.5-预测及评估" data-toc-modified-id="2.5-预测及评估-2.5">2.5 预测及评估</a></span></li></ul></li><li><span><a href="#3-总结：" data-toc-modified-id="3-总结：-3">3 总结：</a></span></li></ul></div>

Action2:使用DeepFM对movielens进行评分预测    

1、使用DeepFM工具进行评分预测，代码正确，可以跑通简单的movielens_sample数据集（10points）    

2、可以跑通完整的movielens数据集（10points）

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder  # 标签数值化
from sklearn.metrics import mean_squared_error  # MSE
from sklearn.model_selection import train_test_split # 拆分数据集

In [10]:
from deepctr.models import DeepFM
from deepctr.feature_column import get_feature_names, SparseFeat

# 1 movielens_sample数据集

## 1.1 数据载入

In [11]:
df = pd.read_csv('./movielens_sample.txt')

In [12]:
df.shape

(200, 10)

In [13]:
df.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,3299,235,4,968035345,Ed Wood (1994),Comedy|Drama,F,25,4,19119
1,3630,3256,3,966536874,Patriot Games (1992),Action|Thriller,M,18,4,77005


## 1.2 数据预处理

In [14]:
# 稀疏特征和目标值
sparse_features = ["user_id", "movie_id", "gender", "age", "occupation", "zip"]
target = ['rating']

### 1.2.1 标签数值化

In [15]:
for f in sparse_features:
    transfer = LabelEncoder()
    df[f] = transfer.fit_transform(df[f])

In [16]:
df.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,107,12,4,968035345,Ed Wood (1994),Comedy|Drama,0,2,4,35
1,123,169,3,966536874,Patriot Games (1992),Action|Thriller,1,1,4,118


### 1.2.2 生成词向量

In [17]:
# df[f].nunique() 计算每个特征中不同特征值的个数
fixlen_feature_columns = [SparseFeat(f, df[f].nunique()) for f in sparse_features]

In [18]:
type(fixlen_feature_columns[0])

deepctr.feature_column.SparseFeat

In [19]:
print(fixlen_feature_columns[0])

SparseFeat(name='user_id', vocabulary_size=193, embedding_dim=4, use_hash=False, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x7efe7e8673c8>, embedding_name='user_id', group_name='default_group', trainable=True)


In [20]:
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns

In [21]:
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [22]:
type(feature_names), len(feature_names)

(list, 6)

In [23]:
feature_names

['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']

## 1.3 拆分数据集

In [24]:
x_train, x_test, y_train, y_test = train_test_split(df[feature_names], 
                                                    df['rating'], 
                                                    test_size=0.2, 
                                                    random_state=30)
x_train.shape, x_test.shape

((160, 6), (40, 6))

In [25]:
x_train_dict = {name: x_train[name].values for name in feature_names}

In [26]:
x_test_dict = {name: x_test[name].values for name in feature_names}

## 1.4 使用DeepFM训练

### 1.4.1 实例化模型

In [27]:
model = DeepFM(linear_feature_columns=linear_feature_columns,  # 包含模型线性部分所使用的所有特征的迭代器
               dnn_feature_columns=dnn_feature_columns,  # 一个包含模型深层部分使用的所有特征的迭代器
               task='regression' # 回归任务
              )

### 1.4.2 编译模型

In [28]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

### 1.4.3 训练模型

In [29]:
y_train.shape

(160,)

In [30]:
y_train = y_train.values.reshape((-1, 1))
y_test = y_test.values.reshape(-1, 1)

In [31]:
historys = model.fit(x_train_dict, y_train, 
                    batch_size=256, 
                    epochs=50, 
                    verbose=0, # =1 代表进度条
                    validation_split=0.2 # 验证集拆分比例
                   )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [32]:
pred = model.predict(x_test_dict, batch_size=256)

In [33]:
mse = mean_squared_error(y_test, pred)
mse

1.080217266408323

In [34]:
rmse = mse ** 0.5
rmse

1.0393350116340365

# 2 movielens数据集

## 2.1 读入数据

In [35]:
datas = pd.read_csv('ratings.csv')

In [36]:
datas

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
...,...,...,...,...
1048570,7120,168,5.0,1175543061
1048571,7120,253,4.0,1175542225
1048572,7120,260,5.0,1175542035
1048573,7120,261,4.0,1175543376


## 2.2 预处理

### 2.2.1 标签数值化

In [37]:
sparse_features = ['userId', 'movieId', 'timestamp']
target = ['rating']

In [38]:
for f in  sparse_features:
    transfor = LabelEncoder()
    datas[f] = transfor.fit_transform(datas[f])

In [39]:
datas.head(2)

,userId,movieId,rating,timestamp
0,0,1,3.5,340880
1,0,28,3.5,340785


### 2.2.2生成词向量

In [40]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=datas[feat].nunique()) for feat in sparse_features]

In [41]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(dnn_feature_columns + linear_feature_columns)

In [42]:
feature_names

['userId', 'movieId', 'timestamp']

## 2.3 拆分数据集并为模型生成输入数据

In [43]:
x_train, x_test, y_train, y_test = train_test_split(datas[feature_names], 
                                                    datas[target], 
                                                    test_size=0.2, 
                                                    random_state=30)

In [44]:
# 特征和目标值均为DataFrame
x_train.shape, y_train.shape

((838860, 3), (838860, 1))

In [45]:
train_model_input = {name:x_train[name].values for name in feature_names}
test_model_input = {name:x_test[name].values for name in feature_names}

In [46]:
train_model_input

{'movieId': array([  46, 2646, 6917, ..., 1861, 1076, 1286]),
 'timestamp': array([ 47112,  90860, 737376, ...,  87251, 109057, 679117]),
 'userId': array([2719,  677, 3283, ..., 3853, 6649, 2932])}

## 2.4 用DeepFM建模

In [47]:
model = DeepFM(linear_feature_columns, 
               dnn_feature_columns,
               task='regression'
              )

In [48]:
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [50]:
%%time
history = model.fit(train_model_input, 
                    y_train.values, 
                    batch_size=256, 
                    epochs=10, 
                    verbose=0,
                    validation_split=0.2)

CPU times: user 18min 25s, sys: 33.5 s, total: 18min 59s
Wall time: 11min 17s


## 2.5 预测及评估

In [51]:
preds = model.predict(test_model_input, batch_size=256)

In [53]:
mse = mean_squared_error(y_test.values, preds)
mse

0.8357133121930652

In [55]:
rmse = mse ** 0.5
rmse

0.9141735678704921

# 3 总结：

1. DeepFM的模型的方法及参数和TensorFlow很相似；
2. 由于DeepFM的神经网络部分是基于TensorFlow，所以送入模型的y必须是二维的；
3. 与Action1的用libFM训练的FM模型相比，DeepFM的RMSE更小些（libFM RMSE:1.344，DeepFM RMSE:0.914）。